In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Employment Services including Better Jobs Ontario'

url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=7&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
data = []

# ---- Data Extraction ----
for record in results_container.find_all('div', class_='record-list-row'):
    # Service Name & Link
    name_tag = record.find('a', class_='record-agency')
    service_name = name_tag.text.strip() if name_tag else ""
    record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

    # Organization
    org_tag = record.find('div', class_='subtitle')
    organization = org_tag.text.strip() if org_tag else ""

    # Phones
    phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
    phone = [tag.text.strip() for tag in phone_tags]
    phone_numbers = ", ".join(phone) if phone else ""

    # Website
    website_tag = record.find('a', class_='linkexternal_fa6')
    website = website_tag['href'].strip() if website_tag else ""

    # Address
    address_tag = record.find('div', class_='record-info')
    if address_tag:
        address_lines = [line.strip() for line in address_tag.stripped_strings]
        address = ", ".join(address_lines)
    else:
        address = ""

    # ---- Append Row ----
    data.append({
        "Organization": organization,
        "Service Name": service_name,
        "Category": category,
        "Subcategory": subcategory,
        "City": city,
        "Phone Numbers": phone_numbers,
        "Address": address,
        "Website": website,
        "Record Link": record_link
    })

# ---- Convert to DataFrame ----
df = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,HOPE Start,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,519-823-9333,"25 Laurine Ave, Guelph, ON, N1E 4N1 (18km)",https://hopestart.ca/,https://feat.findhelp.ca/record/89429089/?sear...
2,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
3,Starling Community Services,Integrated Employment Services - Guelph,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"1-866-321-4141, 519-822-4141","89 Wyndham St N, 3rd Flr, Guelph, ON, N1H 4E9 ...",https://starlingcs.ca/employment,https://feat.findhelp.ca/record/92981097/?sear...
4,Second Chance Employment Counselling,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,519-823-2440,"177 Norfolk St, Guelph, ON, N1H 4K1 (20km)",https://2ndchance.ca/,https://feat.findhelp.ca/record/72401487/?sear...


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Organization   50 non-null     object
 1   Service Name   50 non-null     object
 2   Category       50 non-null     object
 3   Subcategory    50 non-null     object
 4   City           50 non-null     object
 5   Phone Numbers  50 non-null     object
 6   Address        50 non-null     object
 7   Website        50 non-null     object
 8   Record Link    50 non-null     object
dtypes: object(9)
memory usage: 3.6+ KB


# Looping through 8 pages:


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Employment Services including Better Jobs Ontario'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=7&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Loop through 8 pages ----
for page in range(1, 9):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break  # No results on this page

    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,HOPE Start,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,519-823-9333,"25 Laurine Ave, Guelph, ON, N1E 4N1 (18km)",https://hopestart.ca/,https://feat.findhelp.ca/record/89429089/?sear...
2,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
3,Starling Community Services,Integrated Employment Services - Guelph,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"1-866-321-4141, 519-822-4141","89 Wyndham St N, 3rd Flr, Guelph, ON, N1H 4E9 ...",https://starlingcs.ca/employment,https://feat.findhelp.ca/record/92981097/?sear...
4,Second Chance Employment Counselling,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,519-823-2440,"177 Norfolk St, Guelph, ON, N1H 4K1 (20km)",https://2ndchance.ca/,https://feat.findhelp.ca/record/72401487/?sear...


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Organization   400 non-null    object
 1   Service Name   400 non-null    object
 2   Category       400 non-null    object
 3   Subcategory    400 non-null    object
 4   City           400 non-null    object
 5   Phone Numbers  400 non-null    object
 6   Address        400 non-null    object
 7   Website        400 non-null    object
 8   Record Link    400 non-null    object
dtypes: object(9)
memory usage: 28.3+ KB


In [18]:
df1 = df.copy()

### Pulling info of Ontario Bridge Training Program

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Ontario Bridge Training Program'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=9&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Loop through 8 pages ----
for page in range(1, 9):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break  # No results on this page

    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df2 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df2.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,COSTI,Ontario Bridge Training Program,Looking for Work,Ontario Bridge Training Program,Acton,289-843-3228,"6750 Winston Churchill Blvd, Unit 8A, Mississa...",https://costideb.costi.ca/costi/course/index.p...,https://feat.findhelp.ca/record/90908367/?sear...
1,YMCA of Greater Toronto - Newcomers,"Ontario Bridge Training Program, Meadowale Tow...",Looking for Work,Ontario Bridge Training Program,Acton,416-277-3847,"6677 Meadowvale Town Centre Circle, Suite 15, ...",https://newcomersincanada.ca/services-for-newc...,https://feat.findhelp.ca/record/84034800/?sear...
2,Humber Polytechnic,Ontario Bridge Training Program,Looking for Work,Ontario Bridge Training Program,Acton,416-675-3111,"Humber North Campus, 205 Humber College Blvd, ...",https://humber.ca/community/new-to-canada/brid...,https://feat.findhelp.ca/record/80162146/?sear...
3,Humber Polytechnic,Occupation-Specific Language Training,Looking for Work,Ontario Bridge Training Program,Acton,416-675-6622,"Humber North Campus, 205 Humber College Blvd, ...",https://humber.ca/community/new-to-canada/lang...,https://feat.findhelp.ca/record/89726365/?sear...
4,YMCA of Greater Toronto - Newcomers,"Ontario Bridge Training Program, Etobicoke Alb...",Looking for Work,Ontario Bridge Training Program,Acton,416-277-3847,"Albion Centre, 1530 Albion Rd, Unit 83, Toront...",https://newcomersincanada.ca/services-for-newc...,https://feat.findhelp.ca/record/84034788/?sear...


###  Ontario Employment Assistance Services 

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Ontario Employment Assistance Services'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=10&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data3 = []

# ---- Loop through 97 pages ----
for page in range(1, 98):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break

    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data3.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df3 = pd.DataFrame(data3, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df3.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Adult Learning Centre,Literacy and Basic Skills,Looking for Work,Ontario Employment Assistance Services,Acton,905-873-2200,"Halton Hills Public Library; 17 River St, Halt...",http://www.alnh.ca,https://feat.findhelp.ca/record/94527394/?sear...
1,Adult Learning Centre,Essential Skills for Employment,Looking for Work,Ontario Employment Assistance Services,Acton,905-873-2200,"Halton Hills Public Library; 17 River St, Halt...",http://www.alnh.ca,https://feat.findhelp.ca/record/94527395/?sear...
2,,Downtown Acton Business Improvement Area,Looking for Work,Ontario Employment Assistance Services,Acton,519-853-9555,"Acton Town Hall Centre; 19 Willow St N, Halton...",http://downtownacton.ca,https://feat.findhelp.ca/record/94376183/?sear...
3,Halton Healthcare,"North Halton Mental Health Clinic, Acton",Looking for Work,Ontario Employment Assistance Services,Acton,"1-833-256-7812, 519-853-9741, 1-877-825-9011, ...","19 Willow St N, Halton Hills, ON, L7J 1Z6",http://www.haltonhealthcare.on.ca/services_/28...,https://feat.findhelp.ca/record/94446124/?sear...
4,Community Development Halton,Volunteer Halton,Looking for Work,Ontario Employment Assistance Services,Acton,"1-855-395-8807, 905-632-1975","The Roxy Centre; 6 Mill St E, Halton Hills, ON...",http://www.cdhalton.ca/learning-engagement,https://feat.findhelp.ca/record/93937498/?sear...


In [21]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4850 entries, 0 to 4849
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Organization   4850 non-null   object
 1   Service Name   4850 non-null   object
 2   Category       4850 non-null   object
 3   Subcategory    4850 non-null   object
 4   City           4850 non-null   object
 5   Phone Numbers  4850 non-null   object
 6   Address        4850 non-null   object
 7   Website        4850 non-null   object
 8   Record Link    4850 non-null   object
dtypes: object(9)
memory usage: 341.1+ KB


### Ontario Job Bank

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Ontario Job Bank'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=11&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data5 = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data5.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df5 = pd.DataFrame(data5, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df5.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Canada. Employment and Social Development Canada,Job Bank,Looking for Work,Ontario Job Bank,Acton,,Recommended,https://www.jobbank.gc.ca,https://feat.findhelp.ca/record/70211081/?sear...


### Skills Development Fund (SDF)

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Skills Development Fund (SDF)'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=70&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data6 = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data6.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df6 = pd.DataFrame(data6, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df6.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Support for Employers,Skills Development Fund (SDF),Acton,"1-800-387-5656, 416-326-5656",Recommended,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...


### Supported Employment

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Supported Employment'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=12&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data7 = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data7.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df7 = pd.DataFrame(data7, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df7.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Community Living Mississauga,Career Connection,Looking for Work,Supported Employment,Acton,905-542-2694,"6695 Millcreek Dr, Unit 1, Mississauga, ON, L5...",https://career-connection.ca,https://feat.findhelp.ca/record/72030970/?sear...
1,"Bennett Edge, The - Peel Region",Integrated Employment Services,Looking for Work,Supported Employment,Acton,"905-459-4060, 905-459-4080","7900 Hurontario St, Suite 202, Brampton, ON, L...",https://the-bennett-edge-8b6547e67a54951d40819...,https://feat.findhelp.ca/record/72388168/?sear...
2,March of Dimes Canada (GTA),"Integrated Employment Services, Mississauga",Looking for Work,Supported Employment,Acton,"1-877-427-6990, 905-607-3463","2227 South Millway, Suite 201, Mississauga, ON...",https://www.modcemploymentservices.ca,https://feat.findhelp.ca/record/88785681/?sear...
3,Community Living Hamilton,Integrated Employment Services,Looking for Work,Supported Employment,Acton,905-528-0281,"191 York Blvd, Hamilton, ON, L8R 1Y6 (44km)",https://communitylivinghamilton.com/services/#...,https://feat.findhelp.ca/record/79755391/?sear...
4,PATH Employment Services,Integrated Employment Services,Looking for Work,Supported Employment,Acton,905-528-6611,"12 James St N #501, Hamilton, ON, L8P 1A1 (44km)",http://www.pathemployment.com,https://feat.findhelp.ca/record/64671819/?sear...


### Youth Job Connection 


In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Youth Job Connection'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=13&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data8 = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data8.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df8 = pd.DataFrame(data8, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df8.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Mamaweswen, The North Shore Tribal Council",Ontario Employment Services,Looking for Work,Youth Job Connection,Acton,"1-877-633-7558, 705-844-2340","473 Hwy 17, Serpent River First Nation, ON, P0...",https://mamaweswen.com/,https://feat.findhelp.ca/record/65276341/?sear...


### Youth Job Connection Summer

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Youth Job Connection Summer'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=14&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data9 = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data9.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df9 = pd.DataFrame(data9, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df9.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Mamaweswen, The North Shore Tribal Council",Ontario Employment Services,Looking for Work,Youth Job Connection Summer,Acton,"1-877-633-7558, 705-844-2340","473 Hwy 17, Serpent River First Nation, ON, P0...",https://mamaweswen.com/,https://feat.findhelp.ca/record/65276341/?sear...


### Ontario Job Creation Partnerships 

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Looking for Work'
subcategory = 'Ontario Job Creation Partnerships'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=15&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data10 = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data10.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df10 = pd.DataFrame(data10, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df10.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Ontario Job Creation Partnerships,Looking for Work,Ontario Job Creation Partnerships,Acton,"1-800-387-5656, 416-326-5656","400 University Ave, 3rd Fl, Toronto, ON, M7A 1...",https://www.ontario.ca/page/ontario-job-creati...,https://feat.findhelp.ca/record/93082636/?sear...


# Training and Upgrading =


###	Better Jobs Ontario 

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Better Jobs Ontario'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=26&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data11 = []

# ---- Loop through 9 pages ----
for page in range(1, 10):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break

    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data11.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df11 = pd.DataFrame(data11, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df11.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Training and Upgrading,Better Jobs Ontario,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,HOPE Start,Integrated Employment Services,Training and Upgrading,Better Jobs Ontario,Acton,519-823-9333,"25 Laurine Ave, Guelph, ON, N1E 4N1 (18km)",https://hopestart.ca/,https://feat.findhelp.ca/record/89429089/?sear...
2,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Training and Upgrading,Better Jobs Ontario,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
3,Starling Community Services,Integrated Employment Services - Guelph,Training and Upgrading,Better Jobs Ontario,Acton,"1-866-321-4141, 519-822-4141","89 Wyndham St N, 3rd Flr, Guelph, ON, N1H 4E9 ...",https://starlingcs.ca/employment,https://feat.findhelp.ca/record/92981097/?sear...
4,Second Chance Employment Counselling,Integrated Employment Services,Training and Upgrading,Better Jobs Ontario,Acton,519-823-2440,"177 Norfolk St, Guelph, ON, N1H 4K1 (20km)",https://2ndchance.ca/,https://feat.findhelp.ca/record/72401487/?sear...


In [29]:
df11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Organization   450 non-null    object
 1   Service Name   450 non-null    object
 2   Category       450 non-null    object
 3   Subcategory    450 non-null    object
 4   City           450 non-null    object
 5   Phone Numbers  450 non-null    object
 6   Address        450 non-null    object
 7   Website        450 non-null    object
 8   Record Link    450 non-null    object
dtypes: object(9)
memory usage: 31.8+ KB


### Adult High School Credit Program 


In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Adult High School Credit Program'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=27&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data12 = []

# ---- Loop through 3 pages ----
for page in range(1, 4):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break

    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""
        address_tag = record.find('div', class_='record-info')
        address = ", ".join([line.strip() for line in address_tag.stripped_strings]) if address_tag else ""

        data12.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df12 = pd.DataFrame(data12, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df12.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Halton District School Board,Adult High School Diploma Program,Training and Upgrading,Adult High School Credit Program,Acton,905-632-2944,"Gary Allan Learning Centre, Halton Hills Campu...",http://garyallan.ca,https://feat.findhelp.ca/record/94599663/?sear...
1,Halton Catholic District School Board - Thomas...,Centre for Adult and Continuing Education,Training and Upgrading,Adult High School Credit Program,Acton,905-849-7555,"2800 High Point Dr, 2nd Fl, Milton, ON, L9T 6P...",http://merton.hcdsb.org,https://feat.findhelp.ca/record/94526958/?sear...
2,Halton Catholic District School Board - Thomas...,Adult High School Diploma Program,Training and Upgrading,Adult High School Credit Program,Acton,905-849-7555,"2800 High Point Dr, 2nd Fl, Milton, ON, L9T 6P...",http://merton.hcdsb.org/adult-students/earn-yo...,https://feat.findhelp.ca/record/94526963/?sear...
3,Upper Grand District School Board Main,"Adult High School Diploma Program, Guelph Well...",Training and Upgrading,Adult High School Credit Program,Acton,"1-800-321-4025, 519-836-7280","1428 Gordon St, Guelph, ON, N1L 1C8 (18km)",https://www.ugdsb.ca/continuing-education/,https://feat.findhelp.ca/record/72401350/?sear...
4,Halton District School Board,Adult High School Diploma Program,Training and Upgrading,Adult High School Credit Program,Acton,905-632-2944,"Gary Allan Learning Centre, Milton Campus; 215...",http://garyallan.ca,https://feat.findhelp.ca/record/94599669/?sear...


### Canadian Adult Education Credential (CAEC)

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Canadian Adult Education Credential (CAEC)'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=67&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Loop through 3 pages ----
for page in range(1, 4):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break  # No results on this page

    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df10 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df10.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Halton District School Board,Adult High School Diploma Program,Training and Upgrading,Canadian Adult Education Credential (CAEC),Acton,905-632-2944,"Gary Allan Learning Centre, Halton Hills Campu...",http://garyallan.ca,https://feat.findhelp.ca/record/94599663/?sear...
1,Halton Catholic District School Board - Thomas...,Centre for Adult and Continuing Education,Training and Upgrading,Canadian Adult Education Credential (CAEC),Acton,905-849-7555,"2800 High Point Dr, 2nd Fl, Milton, ON, L9T 6P...",http://merton.hcdsb.org,https://feat.findhelp.ca/record/94526958/?sear...
2,Halton Catholic District School Board - Thomas...,Adult High School Diploma Program,Training and Upgrading,Canadian Adult Education Credential (CAEC),Acton,905-849-7555,"2800 High Point Dr, 2nd Fl, Milton, ON, L9T 6P...",http://merton.hcdsb.org/adult-students/earn-yo...,https://feat.findhelp.ca/record/94526963/?sear...
3,Upper Grand District School Board Main,"Adult High School Diploma Program, Guelph Well...",Training and Upgrading,Canadian Adult Education Credential (CAEC),Acton,"1-800-321-4025, 519-836-7280","1428 Gordon St, Guelph, ON, N1L 1C8 (18km)",https://www.ugdsb.ca/continuing-education/,https://feat.findhelp.ca/record/72401350/?sear...
4,Halton District School Board,Adult High School Diploma Program,Training and Upgrading,Canadian Adult Education Credential (CAEC),Acton,905-632-2944,"Gary Allan Learning Centre, Milton Campus; 215...",http://garyallan.ca,https://feat.findhelp.ca/record/94599669/?sear...


### Literacy and Basic Skills 

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Literacy and Basic Skills'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=29&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Loop through 8 pages ----
for page in range(1, 9):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})
    if not results_container:
        break  # No results on this page

    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df11 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df11.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Adult Learning Centre,Literacy and Basic Skills,Training and Upgrading,Literacy and Basic Skills,Acton,905-873-2200,"Halton Hills Public Library; 17 River St, Halt...",http://www.alnh.ca,https://feat.findhelp.ca/record/94527394/?sear...
1,Adult Learning Centre,Literacy and Basic Skills,Training and Upgrading,Literacy and Basic Skills,Acton,905-873-2200,"72 Mill St, Suite 206, Halton Hills, ON, L7G 2...",http://www.alnh.ca,https://feat.findhelp.ca/record/94527403/?sear...
2,Halton Catholic District School Board - Thomas...,Centre for Adult and Continuing Education,Training and Upgrading,Literacy and Basic Skills,Acton,905-849-7555,"2800 High Point Dr, 2nd Fl, Milton, ON, L9T 6P...",http://merton.hcdsb.org,https://feat.findhelp.ca/record/94526958/?sear...
3,Upper Grand District School Board Main,Continuing Education - Skills Upgrading Progra...,Training and Upgrading,Literacy and Basic Skills,Acton,519-836-7280,"1428 Gordon St, Guelph, ON, N1L 1C8 (18km)",https://www.ugdsb.ca/continuing-education/,https://feat.findhelp.ca/record/72401348/?sear...
4,Upper Grand District School Board Main,Continuing Education - Skills Upgrading Progra...,Training and Upgrading,Literacy and Basic Skills,Acton,519-836-7280,"Tytler School, 131 Ontario St, Guelph, ON, N1E...",https://www.ugdsb.ca/o/cep/page/skills-upgradi...,https://feat.findhelp.ca/record/72401367/?sear...


### Literacy and Basic Skills Online 

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Literacy and Basic Skills Online'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=32&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Single page scrape ----
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df12 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df12.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",e-Channel,Training and Upgrading,Literacy and Basic Skills Online,Acton,,,https://www.tcu.gov.on.ca/eng/eopg/programs/lb...,https://feat.findhelp.ca/record/69796409/?sear...
1,George Brown College,Academic Upgrading Program for Deaf and Hard o...,Training and Upgrading,Literacy and Basic Skills Online,Acton,,"School of Work and College Preparation, 341 Ki...",https://www.georgebrown.ca/deafupgrading,https://feat.findhelp.ca/record/69801129/?sear...
2,George Brown College,Deaf Learn Now,Training and Upgrading,Literacy and Basic Skills Online,Acton,,"School of Work and College Preparation, 341 Ki...",https://deaflearnnow.ca/,https://feat.findhelp.ca/record/75198300/?sear...
3,,The LearningHUB,Training and Upgrading,Literacy and Basic Skills Online,Acton,1-844-470-7877,,https://www.learninghub.ca/,https://feat.findhelp.ca/record/71096113/?sear...
4,Saugeen First Nation,Literacy and Basic Skills,Training and Upgrading,Literacy and Basic Skills Online,Acton,519-797-5986,Saugeen Enjikendasswiing Zhaang Adult Learning...,https://saugeenfirstnation.com/public-library,https://feat.findhelp.ca/record/71097470/?sear...


### Literacy and Basic Skills Support Organizations

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Literacy and Basic Skills Support Organizations'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=69&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Single page scrape ----
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df13 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df13.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Peel-Halton-Dufferin Adult Learning Network,Literacy and Basic Skills Support Program,Training and Upgrading,Literacy and Basic Skills Support Organizations,Acton,905-812-3533,"151 Superior Blvd, Units 19-23, Mississauga, O...",http://www.phdaln.on.ca/,https://feat.findhelp.ca/record/69806829/?sear...
1,Laubach Literacy Ontario,Literacy and Basic Skills,Training and Upgrading,Literacy and Basic Skills Support Organizations,Acton,519-743-3309,"8A-65 Noecker St, Waterloo, ON, N2J 2R6 (43km)",https://www.laubach-on.ca/,https://feat.findhelp.ca/record/73447076/?sear...
2,,AlphaPlus,Training and Upgrading,Literacy and Basic Skills Support Organizations,Acton,"1-800-788-1120, 416-322-1012","120 Eglinton Ave E, Suite 500, Toronto, ON, M4...",https://alphaplus.ca,https://feat.findhelp.ca/record/69800015/?sear...
3,Metro Toronto Movement for Literacy,Literacy and Basic Skills Support Organization,Training and Upgrading,Literacy and Basic Skills Support Organizations,Acton,416-961-5557,,https://www.mtml.ca,https://feat.findhelp.ca/record/69795853/?sear...
4,,College Sector Committee for Adult Upgrading,Training and Upgrading,Literacy and Basic Skills Support Organizations,Acton,416-639-1525,"Colleges Ontario; 130 Queens Quay E, Suite 606...",http://www.cscau.com,https://feat.findhelp.ca/record/69799460/?sear...


### Ontario Bridge Training Program 

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Ontario Bridge Training Program'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=30&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Single page scrape ----
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df14 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df14.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,COSTI,Ontario Bridge Training Program,Training and Upgrading,Ontario Bridge Training Program,Acton,289-843-3228,"6750 Winston Churchill Blvd, Unit 8A, Mississa...",https://costideb.costi.ca/costi/course/index.p...,https://feat.findhelp.ca/record/90908367/?sear...
1,YMCA of Greater Toronto - Newcomers,"Ontario Bridge Training Program, Meadowale Tow...",Training and Upgrading,Ontario Bridge Training Program,Acton,416-277-3847,"6677 Meadowvale Town Centre Circle, Suite 15, ...",https://newcomersincanada.ca/services-for-newc...,https://feat.findhelp.ca/record/84034800/?sear...
2,Humber Polytechnic,Ontario Bridge Training Program,Training and Upgrading,Ontario Bridge Training Program,Acton,416-675-3111,"Humber North Campus, 205 Humber College Blvd, ...",https://humber.ca/community/new-to-canada/brid...,https://feat.findhelp.ca/record/80162146/?sear...
3,Humber Polytechnic,Occupation-Specific Language Training,Training and Upgrading,Ontario Bridge Training Program,Acton,416-675-6622,"Humber North Campus, 205 Humber College Blvd, ...",https://humber.ca/community/new-to-canada/lang...,https://feat.findhelp.ca/record/89726365/?sear...
4,YMCA of Greater Toronto - Newcomers,"Ontario Bridge Training Program, Etobicoke Alb...",Training and Upgrading,Ontario Bridge Training Program,Acton,416-277-3847,"Albion Centre, 1530 Albion Rd, Unit 83, Toront...",https://newcomersincanada.ca/services-for-newc...,https://feat.findhelp.ca/record/84034788/?sear...


### Skills Development Fund (SDF)


In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Skills Development Fund (SDF)'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=68&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Single page scrape ----
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df15 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df15.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Training and Upgrading,Skills Development Fund (SDF),Acton,"1-800-387-5656, 416-326-5656",Recommended,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...


# Apprenticeship ### 	Apprenticeship Program 

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Apprenticeship'
subcategory = 'Apprenticeship Program'
url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=2&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Single page scrape ----
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        # Service Name & Link
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        # Organization
        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        # Phones
        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        # Website
        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        # Address
        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        # ---- Append Row ----
        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

# ---- Convert to DataFrame ----
df16 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df16.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Skilled Trades Ontario,Apprenticeship Program,Apprenticeship,Apprenticeship Program,Acton,"855-299-0028, 647-847-3000",,https://www.skilledtradesontario.ca,https://feat.findhelp.ca/record/93826209/?sear...


### Ontario Youth Apprenticeship Program (OYAP) 

In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Apprenticeship'
subcategory = 'Ontario Youth Apprenticeship Program (OYAP)'

base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=16&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

for page in range(1, 3):  # Loop through pages 1 and 2
    url = f"{base_url}&page={page}"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })
    else:
        print(f"No results container found on page {page}")

df17 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df17.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Upper Grand District School Board Main,Ontario Youth Apprenticeship Program,Apprenticeship,Ontario Youth Apprenticeship Program (OYAP),Acton,"1-800-321-4025, 519-766-5960","500 Victoria Rd N, Guelph, ON, N1E 6K2 (19km)",https://www.oyappajo.com/,https://feat.findhelp.ca/record/72401201/?sear...
1,Ernest C Drury School for the Deaf,Ontario Youth Apprenticeship Program,Apprenticeship,Ontario Youth Apprenticeship Program (OYAP),Acton,905-878-2851,Ernest C Drury School for the Deaf; 255 Ontari...,http://oyap.com,https://feat.findhelp.ca/record/93939586/?sear...
2,Wellington Catholic District School Board,Ontario Youth Apprenticeship Program,Apprenticeship,Ontario Youth Apprenticeship Program (OYAP),Acton,519-836-2170,"54 Westmount Rd, Guelph, ON, N1H 5H7 (21km)",https://www.oyappajo.com/contact/,https://feat.findhelp.ca/record/72401447/?sear...
3,Wellington Catholic District School Board,Ontario Youth Apprenticeship Program,Apprenticeship,Ontario Youth Apprenticeship Program (OYAP),Acton,519-836-2170,"255 Speedvale Ave W, Guelph, ON, N1H 1C5 (22km)",https://www.oyappajo.com/contact/,https://feat.findhelp.ca/record/74557741/?sear...
4,,Career Education Council,Apprenticeship,Ontario Youth Apprenticeship Program (OYAP),Acton,519-827-6468,"17A-218 Silvercreek Pkwy N, Suite 1147, Guelph...",http://www.careereducationcouncil.ca/,https://feat.findhelp.ca/record/72104358/?sear...


### Pre-Apprenticeship 

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Apprenticeship'
subcategory = 'Pre-Apprenticeship'

base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=17&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

for page in range(1, 3):  # Loop through pages 1 and 2
    url = f"{base_url}&page={page}"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })
    else:
        print(f"No results container found on page {page}")

df18 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df18.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Pre-Apprenticeship Training Program,Apprenticeship,Pre-Apprenticeship,Acton,1-800-387-5656,"400 University Ave, 3rd Fl, Toronto, ON, M7A 1...",https://www.tcu.gov.on.ca/eng/apprentices/pre_...,https://feat.findhelp.ca/record/93082646/?sear...
1,"Ontario. Ministry of Labour, Immigration, Trai...",Pre-Apprenticeship Training Program,Apprenticeship,Pre-Apprenticeship,Acton,1-800-387-5656,"400 University Ave, 3rd Fl, Toronto, ON, M7A 1...",https://www.tcu.gov.on.ca/eng/apprentices/pre_...,https://feat.findhelp.ca/record/93082646/?sear...


### Skills Development Fund (SDF) 

In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Training and Upgrading'
subcategory = 'Skills Development Fund (SDF)'

base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=65&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

for page in range(1, 3):  # Loop through pages 1 and 2
    url = f"{base_url}&page={page}"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })
    else:
        print(f"No results container found on page {page}")

df19 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df19.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Training and Upgrading,Skills Development Fund (SDF),Acton,"1-800-387-5656, 416-326-5656",,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...
1,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Training and Upgrading,Skills Development Fund (SDF),Acton,"1-800-387-5656, 416-326-5656",,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...


# Support for Employers
### Canada-Ontario Job Grant

In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Canada-Ontario Job Grant'

base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=18&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

for page in range(1, 7):  # Loop through pages 1 to 6
    url = f"{base_url}&page={page}"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })
    else:
        print(f"No results container found on page {page}")

df20 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df20.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Support for Employers,Canada-Ontario Job Grant,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Support for Employers,Canada-Ontario Job Grant,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
2,Starling Community Services,Integrated Employment Services - Guelph,Support for Employers,Canada-Ontario Job Grant,Acton,"1-866-321-4141, 519-822-4141","89 Wyndham St N, 3rd Flr, Guelph, ON, N1H 4E9 ...",https://starlingcs.ca/employment,https://feat.findhelp.ca/record/92981097/?sear...
3,Second Chance Employment Counselling,Integrated Employment Services,Support for Employers,Canada-Ontario Job Grant,Acton,519-823-2440,"177 Norfolk St, Guelph, ON, N1H 4K1 (20km)",https://2ndchance.ca/,https://feat.findhelp.ca/record/72401487/?sear...
4,COSTI,"Integrated Employment Services, Brampton",Support for Employers,Canada-Ontario Job Grant,Acton,905-459-8855,"10 Gillingham Dr, Suite 300, Brampton, ON, L6X...",http://www.costi.org/programs/location_details...,https://feat.findhelp.ca/record/72008633/?sear...


### Employment Services including Better Jobs Ontario 

In [42]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Employment Services including Better Jobs Ontario'

base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=19&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

for page in range(1, 10):  # Loop pages 1 to 9
    url = f"{base_url}&page={page}"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })
    else:
        print(f"No results container found on page {page}")

df21 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df21.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Support for Employers,Employment Services including Better Jobs Ontario,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,HOPE Start,Integrated Employment Services,Support for Employers,Employment Services including Better Jobs Ontario,Acton,519-823-9333,"25 Laurine Ave, Guelph, ON, N1E 4N1 (18km)",https://hopestart.ca/,https://feat.findhelp.ca/record/89429089/?sear...
2,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Support for Employers,Employment Services including Better Jobs Ontario,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
3,Starling Community Services,Integrated Employment Services - Guelph,Support for Employers,Employment Services including Better Jobs Ontario,Acton,"1-866-321-4141, 519-822-4141","89 Wyndham St N, 3rd Flr, Guelph, ON, N1H 4E9 ...",https://starlingcs.ca/employment,https://feat.findhelp.ca/record/92981097/?sear...
4,Second Chance Employment Counselling,Integrated Employment Services,Support for Employers,Employment Services including Better Jobs Ontario,Acton,519-823-2440,"177 Norfolk St, Guelph, ON, N1H 4K1 (20km)",https://2ndchance.ca/,https://feat.findhelp.ca/record/72401487/?sear...


### Integrated Employment Services including Better Jobs Ontario 

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Integrated Employment Services including Better Jobs Ontario'

base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=20&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

for page in range(1, 9):  # Loop pages 1 to 8
    url = f"{base_url}&page={page}"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })
    else:
        print(f"No results container found on page {page}")

df22 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df22.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Support for Employers,Integrated Employment Services including Bette...,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,HOPE Start,Integrated Employment Services,Support for Employers,Integrated Employment Services including Bette...,Acton,519-823-9333,"25 Laurine Ave, Guelph, ON, N1E 4N1 (18km)",https://hopestart.ca/,https://feat.findhelp.ca/record/89429089/?sear...
2,Stride,Integrated Employment Services,Support for Employers,Integrated Employment Services including Bette...,Acton,905-693-4252,"Milton Mall; 55 Ontario St S, Suite 26, Milton...",http://stride.on.ca,https://feat.findhelp.ca/record/93939773/?sear...
3,HMC Connections,Integrated Employment Services,Support for Employers,Integrated Employment Services including Bette...,Acton,905-842-2486,"55 Ontario St S, Unit A3, Milton, ON, L9T 2M3 ...",http://hmcconnections.com/employment-support,https://feat.findhelp.ca/record/94377039/?sear...
4,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Support for Employers,Integrated Employment Services including Bette...,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...


### Local Boards 

In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Local Boards'

url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=21&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df23 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df23.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,,Peel Halton Workforce Development Group,Support for Employers,Local Boards,Acton,905-306-9588,,https://www.peelhaltonworkforce.com/,https://feat.findhelp.ca/record/93111459/?sear...
1,,Workforce Planning Board of Waterloo Wellingto...,Support for Employers,Local Boards,Acton,519-622-7122,"73 Water St N, Unit 407, Cambridge, ON, N1R 7L...",https://www.workforceplanningboard.com/,https://feat.findhelp.ca/record/73443182/?sear...
2,,Toronto Workforce Innovation Group,Support for Employers,Local Boards,Acton,416-934-1653,"Centre for Social Innovation, 192 Spadina Ave,...",https://workforceinnovation.ca/,https://feat.findhelp.ca/record/69800905/?sear...
3,Workforce Planning Board of Grand Erie,Local Board,Support for Employers,Local Boards,Acton,519-756-1116,"1100 Clarence St S, Suite 101, Brantford, ON, ...",https://workforceplanningboard.org/,https://feat.findhelp.ca/record/79835885/?sear...
4,,Workforce Planning Board of York Region,Support for Employers,Local Boards,Acton,,"238 Wellington St E, Suite 209, Aurora, ON, L4...",http://www.wpboard.ca,https://feat.findhelp.ca/record/69802107/?sear...


### Ontario Job Bank 

In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Ontario Job Bank'

url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=22&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df24 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df24.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Canada. Employment and Social Development Canada,Job Bank,Support for Employers,Ontario Job Bank,Acton,,,https://www.jobbank.gc.ca,https://feat.findhelp.ca/record/70211081/?sear...


### Ontario Job Creation Partnerships 

In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Ontario Job Creation Partnerships'

url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=23&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df25 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df25.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Ontario Job Creation Partnerships,Support for Employers,Ontario Job Creation Partnerships,Acton,"1-800-387-5656, 416-326-5656","400 University Ave, 3rd Fl, Toronto, ON, M7A 1...",https://www.ontario.ca/page/ontario-job-creati...,https://feat.findhelp.ca/record/93082636/?sear...


### Skills Development Fund (SDF) 

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Skills Development Fund (SDF)'

url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=66&sd=0&ss=Distance&cf='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

if results_container:
    for record in results_container.find_all('div', class_='record-list-row'):
        name_tag = record.find('a', class_='record-agency')
        service_name = name_tag.text.strip() if name_tag else ""
        record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

        org_tag = record.find('div', class_='subtitle')
        organization = org_tag.text.strip() if org_tag else ""

        phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
        phone = [tag.text.strip() for tag in phone_tags]
        phone_numbers = ", ".join(phone) if phone else ""

        website_tag = record.find('a', class_='linkexternal_fa6')
        website = website_tag['href'].strip() if website_tag else ""

        address_tag = record.find('div', class_='record-info')
        if address_tag:
            address_lines = [line.strip() for line in address_tag.stripped_strings]
            address = ", ".join(address_lines)
        else:
            address = ""

        data.append({
            "Organization": organization,
            "Service Name": service_name,
            "Category": category,
            "Subcategory": subcategory,
            "City": city,
            "Phone Numbers": phone_numbers,
            "Address": address,
            "Website": website,
            "Record Link": record_link
        })

df26 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df26.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Support for Employers,Skills Development Fund (SDF),Acton,"1-800-387-5656, 416-326-5656",Recommended,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...


### Youth Job Connection 

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Youth Job Connection'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=66&sd=0&ss=Distance&cf=&page='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Loop through 3 pages ----
for page_num in range(1, 4):  # Pages 1 to 3
    url = base_url + str(page_num)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })

df27 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df27.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Support for Employers,Youth Job Connection,Acton,"1-800-387-5656, 416-326-5656",Recommended,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...
1,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Support for Employers,Youth Job Connection,Acton,"1-800-387-5656, 416-326-5656",Recommended,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...
2,"Ontario. Ministry of Labour, Immigration, Trai...",Skills Development Fund,Support for Employers,Youth Job Connection,Acton,"1-800-387-5656, 416-326-5656",Recommended,https://www.ontario.ca/page/skills-development...,https://feat.findhelp.ca/record/93081596/?sear...


### Youth Job Connection Summer 

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# ---- Configuration ----
city = 'Acton'
category = 'Support for Employers'
subcategory = 'Youth Job Connection Summer'
base_url = 'https://feat.findhelp.ca/search-results/?searchLocation=Acton&latitude=43.6339088&longitude=-80.041322&topicPath=25&sd=0&ss=Distance&cf=&page='

headers = {"User-Agent": "Mozilla/5.0"}
data = []

# ---- Loop through 3 pages ----
for page_num in range(1, 4):  # Pages 1 to 3
    url = base_url + str(page_num)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    results_container = soup.find('div', id='resultsList', attrs={'role': 'tabpanel'})

    if results_container:
        for record in results_container.find_all('div', class_='record-list-row'):
            name_tag = record.find('a', class_='record-agency')
            service_name = name_tag.text.strip() if name_tag else ""
            record_link = name_tag['href'] if name_tag and name_tag.has_attr('href') else ""

            org_tag = record.find('div', class_='subtitle')
            organization = org_tag.text.strip() if org_tag else ""

            phone_tags = record.find_all('a', href=lambda x: x and x.startswith('tel'))
            phone = [tag.text.strip() for tag in phone_tags]
            phone_numbers = ", ".join(phone) if phone else ""

            website_tag = record.find('a', class_='linkexternal_fa6')
            website = website_tag['href'].strip() if website_tag else ""

            address_tag = record.find('div', class_='record-info')
            if address_tag:
                address_lines = [line.strip() for line in address_tag.stripped_strings]
                address = ", ".join(address_lines)
            else:
                address = ""

            data.append({
                "Organization": organization,
                "Service Name": service_name,
                "Category": category,
                "Subcategory": subcategory,
                "City": city,
                "Phone Numbers": phone_numbers,
                "Address": address,
                "Website": website,
                "Record Link": record_link
            })

df28 = pd.DataFrame(data, columns=[
    "Organization", "Service Name", "Category", "Subcategory", "City",
    "Phone Numbers", "Address", "Website", "Record Link"
])

# ---- Preview ----
df28.head()


,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Support for Employers,Youth Job Connection Summer,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
1,COSTI,"Integrated Employment Services, Brampton",Support for Employers,Youth Job Connection Summer,Acton,905-459-8855,"10 Gillingham Dr, Suite 300, Brampton, ON, L6X...",http://www.costi.org/programs/location_details...,https://feat.findhelp.ca/record/72008633/?sear...
2,COSTI,"Integrated Employment Services, Mississauga",Support for Employers,Youth Job Connection Summer,Acton,905-567-0482,"6750 Winston Churchill Blvd, Unit 8A, Mississa...",https://www.costi.org/programs/location_detail...,https://feat.findhelp.ca/record/69794911/?sear...
3,WCG Services,"Integrated Employment Services, Brampton",Support for Employers,Youth Job Connection Summer,Acton,1-888-353-8501,"134 Main St N, Brampton, ON, L6V 1N8 (23km)",https://eoworks.ca/peel/,https://feat.findhelp.ca/record/81342244/?sear...
4,Community Living Mississauga,Career Connection,Support for Employers,Youth Job Connection Summer,Acton,905-542-2694,"6695 Millcreek Dr, Unit 1, Mississauga, ON, L5...",https://career-connection.ca,https://feat.findhelp.ca/record/72030970/?sear...


# Merge all 28 df:

In [50]:
# List all your DataFrames in order
df_list = [df1, df2, df3, df5, df6, df7, df8, df9, df10,
           df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,
           df21, df22, df23, df24, df25, df26, df27, df28]

# Concatenate all DataFrames vertically
df_acton = pd.concat(df_list, ignore_index=True)

df_acton.head()

,Organization,Service Name,Category,Subcategory,City,Phone Numbers,Address,Website,Record Link
0,VPI Working Solutions,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"1-888-336-9500, 905-873-9816","Halton Hills Shopping Plaza; 235 Guelph St, Un...",http://www.vpi-inc.com,https://feat.findhelp.ca/record/92951011/?sear...
1,HOPE Start,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,519-823-9333,"25 Laurine Ave, Guelph, ON, N1E 4N1 (18km)",https://hopestart.ca/,https://feat.findhelp.ca/record/89429089/?sear...
2,Anishnabeg Outreach Employment and Training,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"519-763-5292, 519-742-0300","400 Stevenson St N, Guelph, ON, N1E 5C3 (19km)",https://www.aocan.org/,https://feat.findhelp.ca/record/72401451/?sear...
3,Starling Community Services,Integrated Employment Services - Guelph,Looking for Work,Employment Services including Better Jobs Ontario,Acton,"1-866-321-4141, 519-822-4141","89 Wyndham St N, 3rd Flr, Guelph, ON, N1H 4E9 ...",https://starlingcs.ca/employment,https://feat.findhelp.ca/record/92981097/?sear...
4,Second Chance Employment Counselling,Integrated Employment Services,Looking for Work,Employment Services including Better Jobs Ontario,Acton,519-823-2440,"177 Norfolk St, Guelph, ON, N1H 4K1 (20km)",https://2ndchance.ca/,https://feat.findhelp.ca/record/72401487/?sear...


In [60]:
df_acton.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7540 entries, 0 to 7539
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Organization   7540 non-null   object
 1   Service Name   7540 non-null   object
 2   Category       7540 non-null   object
 3   Subcategory    7540 non-null   object
 4   City           7540 non-null   object
 5   Phone Numbers  7540 non-null   object
 6   Address        7540 non-null   object
 7   Website        7540 non-null   object
 8   Record Link    7540 non-null   object
dtypes: object(9)
memory usage: 530.3+ KB


In [56]:
df_acton.shape

(7540, 9)

In [53]:
df_acton.to_excel('df_acton_combined.xlsx', index=False)

In [59]:
df_acton['Service Name'].unique()

array(['Integrated Employment Services',
       'Integrated Employment Services - Guelph',
       'Integrated Employment Services, Brampton',
       'Integrated Employment Services, Mississauga',
       'Integrated Employment Services - Meadowvale Town Centre',
       'Career Connection',
       'Integrated Employment Services, Next-Steps Employment Centre, Mississauga-Meadowvale',
       'Integrated Employment Services, Brampton East Location',
       'Integrated Employment Services, Brampton South Location',
       'Integrated Employment Services, Brampton Location',
       'Integrated Employment Services - 40 Peel Centre Dr, Brampton',
       'Integrated Employment Services, Fergus and Wellington County Office',
       'Employment Services',
       'Integrated Employment Services, Malton Location',
       'Integrated Employment Services - 55 City Centre Dr',
       'Integrated Employment Services, Mississauga - Sussex Centre',
       'Integrated Employment Services, Mississauga Loca

In [62]:
# Save to Excel
df_acton.to_excel('df_acton_combined.xlsx', index=False)